In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sys import path
import torch

In [2]:
from hsvi.pytorch import Hierarchy_SVI
from torch.distributions import Normal, Bernoulli

In [3]:
torch.autograd.set_detect_anomaly(True)

In [4]:
torch.random.manual_seed(0)

In [5]:
def softplus(x):
    return torch.log(torch.exp(x)+1.)

## Load and preprocess data

In [6]:
data = pd.read_csv('./review_data.csv')
data.head()

,PaperID,Rev1ID,Rev1Score,Rev2ID,Rev2Score
0,p1,r3,2,r10,3
1,p2,r3,3,r12,3
2,p3,r3,2,r10,4
3,p4,r3,2,r13,3
4,p6,r4,2,r6,3


In [7]:
### form each entry as paper ID, reviewer ID, and score given by the reviewer to the paper ###
reviews = pd.DataFrame(columns=['pid','rid','score'])
reviews.pid = np.repeat(data.PaperID.values,2)
for s in data.PaperID.values:
    reviews.loc[reviews.pid==s,'rid'] = data.loc[data.PaperID==s,['Rev1ID','Rev2ID']].values
    reviews.loc[reviews.pid==s,'score'] = data.loc[data.PaperID==s,['Rev1Score','Rev2Score']].values

### transform paper ID and reviewer ID to numbers ###
reviews.pid = reviews.pid.map(lambda x: int(x[1:])-1)
reviews.rid = reviews.rid.map(lambda x: int(x[1:])-1)

reviews.head()    

,pid,rid,score
0,0,2,2
1,0,9,3
2,1,2,3
3,1,11,3
4,2,2,2


In [8]:
### generate mapping from pid to concecutive ID ###
pid = data.PaperID.map(lambda x: int(x[1:])-1)
id_map = pd.DataFrame(index=pid)
id_map['id'] = data.index.values
id_map

,id
PaperID,
0,0
1,1
2,2
3,3
5,4
...,...
228,214
229,215
230,216


In [9]:
### define hyper-parameters according to the data set ###
S = data.shape[0] #number of submissions
R = reviews.shape[0] #number of reviews
J = len(reviews.rid.unique()) #number of reviewers
T = reviews.score.values.max() #number of score levels
r_per_s = 2 # number of reviews per submission

In [10]:
niter = 500 # number of training iterations
local_iter = 1 # number of local iterations
theta_scale = 1.
score_scale = 0.5

## Define the Reviewer-Bias IRT model

In [11]:
bias = Normal(0.,0.5) # prior of bias
m = reviews.score.mean().astype(np.float32) # empirical mean of score level 
quality = Normal(loc=torch.ones([S])*m,scale=torch.ones([S])) # prior mean set to empirical mean

theta0 = np.array(np.arange(T)+1,ndmin=2,dtype=np.float32)
theta0= torch.from_numpy(np.repeat(theta0,J,axis=0).transpose())

m

3.13242

In [12]:
var_bias_loc = torch.tensor(torch.normal(torch.zeros(J),torch.ones(J)*0.1),requires_grad=True) #torch.randn(J,requires_grad=True)
var_bias_scale = torch.ones(J,requires_grad=True)
#q_bias = Normal(loc=var_bias_loc,scale=softplus(var_bias_scale)*0.1) # posterior of bias

var_quality_loc = torch.tensor(quality.sample(),requires_grad=True)
var_quality_scale = torch.ones([S],requires_grad=True)
#q_quality = Normal(loc=var_quality_loc, scale=softplus(var_quality_scale))
 # generate theta by bias

/home/yu/gits/p36env/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/yu/gits/p36env/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [13]:
def generative_process(var_bias_loc,var_bias_scale,var_quality_loc,var_quality_scale):
    q_bias = Normal(loc=var_bias_loc,scale=softplus(var_bias_scale)) # posterior of bias
    q_quality = Normal(loc=var_quality_loc, scale=softplus(var_quality_scale))

    theta_loc = theta0+q_bias.rsample()
    qs = q_quality.rsample()
    score_quality = qs[torch.tensor(id_map.loc[reviews.pid.values,'id'].values,dtype=torch.long)]
    
    roft = theta_loc[:,torch.tensor(reviews.rid.values,dtype=torch.long)]
    d_loc = score_quality-roft 
    d_scale = torch.ones_like(d_loc)*(np.sqrt((theta_scale**2)+(score_scale**2)))
    d = Normal(d_loc, d_scale)
    y = Bernoulli(1.-d.cdf(torch.zeros_like(d_loc)))
    #y = Bernoulli(1.- normal_cdf(d_loc,d_scale,torch.zeros_like(d_loc)))
    return y, q_bias, q_quality

In [14]:
### generate ovservations of y ###
y_data = np.ones((R,T))*np.arange(T)+1
y_data = (y_data <= reviews.score.values.reshape(-1,1)).astype(dtype=np.float32)
y_data = torch.from_numpy(y_data.transpose())
y_data[:,:5]

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]])

## Define inference method for the model

In [15]:
inference = Hierarchy_SVI(var_dict={'reviewer':[var_bias_loc,var_bias_scale],'paper':[var_quality_loc,var_quality_scale]},learning_rate={'reviewer':0.002,'paper':0.002})

start init hsvi
reviewer KLqp
paper KLqp


## Training process

In [16]:
for _ in range(niter):  
    y, q_bias,q_quality = generative_process(var_bias_loc,var_bias_scale,var_quality_loc,var_quality_scale)
    inference.data = {'reviewer':{y:y_data},'paper':{y:y_data}}
    inference.latent_vars={'reviewer':{bias:q_bias},'paper':{quality:q_quality}}
    for __ in range(local_iter):
        inference.update(scope='paper',retain_graph=True)
        
    loss = inference.update(scope='reviewer',retain_graph=True)
    
    if (_+1)%100==0 or _==0:
        print(' loss {}'.format(loss))
    

/home/yu/gits/p36env/lib/python3.6/site-packages/torch/autograd/__init__.py:149: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


 loss 1.4204634428024292
 loss 0.9012758731842041
 loss 1.079142451286316
 loss 0.7594789266586304
 loss 0.641215980052948
 loss 0.7428992390632629


## Check results

In [17]:
### inferred bias of each reviewer ###
rbias = q_bias.loc.detach()
rb = pd.DataFrame(columns=['RVID','bias'])
rb.RVID = np.argsort(rbias)+1
rb.bias = np.sort(rbias)
rb

,RVID,bias
0,6,-0.676996
1,12,-0.611843
2,5,-0.583088
3,10,-0.532971
4,8,-0.505313
5,1,-0.456541
6,14,-0.444272
7,7,-0.432968
8,2,-0.410278
9,9,-0.329205


In [18]:
### inferred quality of papers ###
quality=q_quality.loc.detach()
qlt = pd.DataFrame(columns=['PID','quality','avg_score'])
qlt.PID = id_map.index.values
qlt.quality = quality
### compare the quality with average score ###
for i in qlt.PID:
    qlt.loc[qlt.PID==i,'avg_score'] = reviews.loc[reviews.pid==i,'score'].mean()
qlt.PID = qlt.PID+1
qlt.head()

,PID,quality,avg_score
0,1,3.127676,2.5
1,2,3.604394,3
2,3,3.241147,3
3,4,2.988182,2.5
4,6,2.806070,2.5
